## 1. Installation

In [ ]:
!pip install ipyplot
!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt
!python setup.py develop
!pip install realesrgan

In [ ]:
import os
import shutil
import cv2
import ipyplot
import matplotlib.pyplot as plt
import glob
import time
import sys
import urllib

from PIL import Image as PILimage

In [ ]:
sys.version

In [ ]:
!nvidia-smi

## 2. Functions

In [ ]:
def comparing_images(image1, image2):
    """
    Displaying two images side by side
    """
    img1 = cv2.cvtColor(cv2.imread(image1), cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(cv2.imread(image2), cv2.COLOR_BGR2RGB)
    
    fig = plt.figure(figsize=(25, 10))
    ax1 = fig.add_subplot(1, 2, 1) 
    plt.title('Initial image: ' + str(image1), fontsize=18, color='Blue')
    ax1.axis('on')
    
    ax2 = fig.add_subplot(1, 2, 2)
    plt.title('GAN image: ' + str(image1), fontsize=18, color='Green')
    ax2.axis('on')
    
    ax1.imshow(img1)
    ax2.imshow(img2)

In [ ]:
def multiple_images(mydir, img_width=300, max_images=15):
    """
    View multiple images from a directory
    """
    img_list = [file for file in os.listdir(mydir)
                if file.endswith(('jpeg', 'png', 'jpg', 'JPEG', 'PNG', 'JPG'))]
    path_img_list = [mydir + '/' + imgfiles for imgfiles in img_list]

    ipyplot.plot_images([PILimage.open(img) for img in path_img_list], 
                        labels=img_list, img_width=img_width, max_images=max_images)

## 3. Input images

In [ ]:
upload_folder = 'inputs/upload'

if not os.path.isdir(upload_folder):
    os.makedirs(upload_folder, exist_ok=True)

## 4. Viewing input images

In [ ]:
img_list = [file for file in os.listdir(upload_folder) if file.endswith(('jpeg', 'png', 'jpg', 'JPG', 'JPEG', 'PNG'))]
img_list

## 5. Running the GAN model

In [ ]:
t1 = time.time()
print("Running the GFPGAN model to restore the images...\n")

print("Deleting any previous results")
!rm -rf results
print("Done")

print("\nRunning the GFPGAN model\n")
!BASICSR_JIT=True python inference_gfpgan.py -i inputs/extract -o results -v 1.4 -s 1 --bg_upsampler realesrgan
print("Done")

print("\nResults:\n")
!ls results/cmp

print("\nDone in", round((time.time() - t1)), "seconds")

## 6. Results

In [ ]:
%%javascript

IPython.OutputArea.auto_scroll_threshold = 9999

In [ ]:
input_folder = 'inputs/upload'  # Initial images
crop_folder = 'results/cropped_faces'  # Faces cropped images
result_folder = 'results/restored_faces'  # Restored faces images
restored_folder = 'results/restored_imgs'  # Full restored images

### 6.1 Viewing crop images

In [ ]:
list_crop = sorted(glob.glob(os.path.join(crop_folder, '*')))
list_result = sorted(glob.glob(os.path.join(result_folder, '*')))

for idx1, idx2 in zip(list_crop, list_result):   
    comparing_images(idx1, idx2)

### 6.2 Viewing restored images

In [ ]:
list_input = sorted(glob.glob(os.path.join(input_folder, '*')))
list_restored = sorted(glob.glob(os.path.join(restored_folder, '*')))

for idx1, idx2 in zip(list_input, list_restored):   
    comparing_images(idx1, idx2)